In [1]:
allegation_df = spark.read.format('csv').options(header='true', inferSchema='true').load('/FileStore/tables/data_allegation.csv')
allegation_df.createOrReplaceTempView("data_allegation")
officerallegation_df = spark.read.format('csv').options(header='true', inferSchema='true').load('/FileStore/tables/data_officerallegation.csv')
officerallegation_df.createOrReplaceTempView("data_officerallegation")
officer_df = spark.read.format('csv').options(header='true', inferSchema='true').load('/FileStore/tables/data_officer.csv')
officer_df.createOrReplaceTempView("data_officer")
salary_df = spark.read.format('csv').options(header='true', inferSchema='true').load('/FileStore/tables/data_salary.csv')
salary_df.createOrReplaceTempView("data_salary")

In [2]:
%sql
SELECT data_salary.officer_id, salary
FROM data_salary
JOIN (
  SELECT officer_id, MAX(year) as year
  FROM data_salary
  GROUP BY officer_id
) most_recent ON most_recent.officer_id=data_salary.id AND most_recent.year=data_salary.year
LIMIT 10 -- For display purpose on Github

officer_id,salary
23453,64662
8370,70260
6788,71700
18102,85584
19677,74946
23008,86142
20463,104628
16400,79980
15172,93648
9523,63456


In [3]:
%sql
WITH allegation_counts AS (
  SELECT officer_id, beat_id, COUNT(allegation_id) as acount
  FROM data_allegation, data_officerallegation, data_officer
  WHERE data_allegation.id=data_officerallegation.allegation_id
  AND data_officer.id=data_officerallegation.officer_id
  GROUP BY officer_id, beat_id
),
recent_salary AS (
  SELECT data_salary.officer_id, salary
  FROM data_salary
  JOIN (
    SELECT officer_id, MAX(year) as year
    FROM data_salary
    GROUP BY officer_id
  ) most_recent ON most_recent.officer_id=data_salary.id AND most_recent.year=data_salary.year
)
SELECT beat_id, ac.officer_id, race, acount
FROM allegation_counts ac
JOIN data_officer o ON o.id = ac.officer_id
JOIN recent_salary s ON s.officer_id = ac.officer_id
WHERE beat_id IS NOT NULL
ORDER BY beat_id, acount DESC
LIMIT 10 -- For display purpose on Github

beat_id,officer_id,race,acount
1,8958,White,2
1,5922,White,1
1,25819,White,1
1,26209,White,1
1,24940,White,1
1,49,Hispanic,1
1,20038,White,1
1,17213,White,1
1,52,Hispanic,1
1,25841,White,1


In [4]:
%sql
WITH allegation_counts AS (
  SELECT officer_id, beat_id, COUNT(allegation_id) as acount
  FROM data_allegation, data_officerallegation, data_officer
  WHERE data_allegation.id=data_officerallegation.allegation_id
  AND data_officer.id=data_officerallegation.officer_id
  GROUP BY officer_id, beat_id
),
recent_salary AS (
  SELECT data_salary.officer_id, salary
  FROM data_salary
  JOIN (
    SELECT officer_id, MAX(year) as year
    FROM data_salary
    GROUP BY officer_id
  ) most_recent ON most_recent.officer_id=data_salary.id AND most_recent.year=data_salary.year
)
SELECT beat_id, ac.officer_id, gender, acount
FROM allegation_counts ac
JOIN data_officer o ON o.id = ac.officer_id
JOIN recent_salary s ON s.officer_id = ac.officer_id
WHERE beat_id IS NOT NULL
ORDER BY beat_id, acount DESC
LIMIT 10 -- For display purpose on Github

beat_id,officer_id,gender,acount
1,8958,M,2
1,5922,M,1
1,25819,M,1
1,26209,M,1
1,17213,M,1
1,24940,F,1
1,20038,M,1
1,49,M,1
1,52,M,1
1,25841,M,1


In [5]:
%sql
WITH allegation_counts AS (
  SELECT officer_id, beat_id, COUNT(allegation_id) as acount
  FROM data_allegation, data_officerallegation, data_officer
  WHERE data_allegation.id=data_officerallegation.allegation_id
  AND data_officer.id=data_officerallegation.officer_id
  GROUP BY officer_id, beat_id
),
recent_salary AS (
  SELECT data_salary.officer_id, salary
  FROM data_salary
  JOIN (
    SELECT officer_id, MAX(year) as year
    FROM data_salary
    GROUP BY officer_id
  ) most_recent ON most_recent.officer_id=data_salary.id AND most_recent.year=data_salary.year
)
SELECT beat_id, ac.officer_id, race, gender, salary, acount
FROM allegation_counts ac
JOIN data_officer o ON o.id = ac.officer_id
JOIN recent_salary s ON s.officer_id = ac.officer_id
WHERE beat_id IS NOT NULL
AND acount > 1
ORDER BY beat_id, acount DESC
LIMIT 10 -- For display purpose on Github

beat_id,officer_id,race,gender,salary,acount
1,8958,White,M,84876,2
10,25841,White,M,92430,2
100,25503,White,M,93354,3
100,13777,Black,M,103590,2
100,2334,White,M,90024,2
100,6620,Hispanic,M,58572,2
101,22269,Hispanic,M,93354,2
102,14442,White,M,87006,7
102,488,Hispanic,M,104628,4
102,4449,White,M,90024,3
